# 🌍 Verra Retirements Explorer
Explore retirements grouped by beneficiary and vintage year using DuckDB. Adapt the queries to go deeper.

In [19]:
# ✅ Install required packages (only once)
!pip install duckdb polars pandas ace_tools

349.72s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [22]:
# 📂 Load dataset and connect to DuckDB
import duckdb
csv_path = 'Verra Data.csv'  # Change if needed
con = duckdb.connect()

# Preview
con.execute(f"SELECT * FROM read_csv_auto('{csv_path}', ignore_errors=True) LIMIT 5").df()

,Issuance Date,Sustainable Development Goals,Vintage Start,Vintage End,ID,Name,Country/Area,Project Type,Methodology,Total Vintage Quantity,Quantity Issued,Serial Number,Additional Certifications,Retirement/Cancellation Date,Retirement Beneficiary,Retirement Reason,Retirement Details
0,2025-05-09,None,2023-01-01,2023-12-31,1949,Longyuan Mulilo De Aar Maanhaarberg Wind Energ...,South Africa,Energy industries (renewable/non-renewable sou...,ACM0002,"304,155","20,000",18079-871311571-871331570-VCS-VCU-814-VER-ZA-1...,None,NaT,None,None,None
1,2025-05-09,None,2020-01-01,2020-12-31,1218,"Evio Kuiñaji Ese´Eja Cuana, To Mitigate Climat...",Peru,Agriculture Forestry and Other Land Use,VM0007,"184,068","87,021",18078-871224550-871311570-VCS-VCU-576-VER-PE-1...,CCB-Biodiversity Gold; CCB-Climate Gold,NaT,None,None,None
2,2025-05-08,None,2020-12-23,2020-12-31,2367,Afforestation in Eucalyptus and Acacia plantat...,Lao,Agriculture Forestry and Other Land Use,AR-ACM0003,"29,728","29,728",18075-871077782-871107509-VCS-VCU-291-VER-LA-1...,None,NaT,None,None,None
3,2025-05-08,None,2021-01-01,2021-12-31,2367,Afforestation in Eucalyptus and Acacia plantat...,Lao,Agriculture Forestry and Other Land Use,AR-ACM0003,"58,520","58,520",18076-871107510-871166029-VCS-VCU-291-VER-LA-1...,None,NaT,None,None,None
4,2025-05-08,None,2022-01-01,2022-12-31,2367,Afforestation in Eucalyptus and Acacia plantat...,Lao,Agriculture Forestry and Other Land Use,AR-ACM0003,"58,520","58,520",18077-871166030-871224549-VCS-VCU-291-VER-LA-1...,None,NaT,None,None,None


In [32]:
query = f"""
SELECT
    EXTRACT(YEAR FROM "Retirement/Cancellation Date") AS retirement_year,
    "Retirement Beneficiary" AS beneficiary,
    "Retirement Reason" AS reason,
    SUM(REPLACE("Quantity Issued", ',', '')::DOUBLE) AS total_volume
FROM read_csv_auto('{csv_path}', ignore_errors=True)
WHERE "Retirement Beneficiary" IS NOT NULL
GROUP BY retirement_year, beneficiary, reason
HAVING retirement_year = 2024
ORDER BY retirement_year, total_volume DESC
"""

top_by_year_df = con.execute(query).df()

# Get top 1 per year (highest volume)
top_1_per_year = (
    top_by_year_df
    .groupby("retirement_year", group_keys=False)
    .apply(lambda x: x.nlargest(100, "total_volume"))
    .reset_index(drop=True)
)

top_1_per_year


/var/folders/x6/5qg3vyj91999h5grpklcyv140000gn/T/ipykernel_42008/2124222387.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(100, "total_volume"))


,retirement_year,beneficiary,reason,total_volume
0,2024,Shell,Environmental Benefit,10930996.0
1,2024,Eni Upstream,Environmental Benefit,3500000.0
2,2024,TASC SA Pty Ltd (COAS account 101046),Section 13 of the South African Carbon Tax Act,2728384.0
3,2024,"Yamato Transport Co., Ltd.",Retirement for Person or Organization,2329701.0
4,2024,Geopost,Environmental Benefit,1620096.0
...,...,...,...,...
95,2024,Lindblad Expeditions,Environmental Benefit,84793.0
96,2024,"Garden of Life, LLC",Retirement for Person or Organization,83333.0
97,2024,Inspire Energy Holdings LLC,Environmental Benefit,81414.0
98,2024,S&P Global Inc.,Environmental Benefit,81075.0


In [13]:
# 🔍 Explore specific beneficiary (e.g., Salesforce)
top_buyer = "Salesforce, Inc."

detail_query = f'''
SELECT
    "Retirement Beneficiary",
    "Name" AS project_name,
    "ID" AS project_id,
    EXTRACT(YEAR FROM "Vintage Start") AS vintage_year,
    REPLACE("Quantity Issued", ',', '')::DOUBLE AS volume
FROM read_csv_auto('{csv_path}', ignore_errors=True)
WHERE "Retirement Beneficiary" = '{top_buyer}'
ORDER BY volume DESC
'''

buyer_projects = con.execute(detail_query).df()
buyer_projects.head(10)

,Retirement Beneficiary,project_name,project_id,vintage_year,volume
0,"Salesforce, Inc.",KUAMUT RAINFOREST CONSERVATION PROJECT,2609,2018,147454.0
1,"Salesforce, Inc.",Delta Blue Carbon – 1,2250,2021,108203.0
2,"Salesforce, Inc.",Nanchang Zhonglan Huanneng Technical Service C...,2358,2021,93637.0
3,"Salesforce, Inc.",Delta Blue Carbon – 1,2250,2021,32988.0
4,"Salesforce, Inc.",Nanchang Zhonglan Huanneng Technical Service C...,2358,2019,28169.0
5,"Salesforce, Inc.",Delta Blue Carbon – 1,2250,2021,24500.0
6,"Salesforce, Inc.",Nanchang Zhonglan Huanneng Technical Service C...,2358,2019,14530.0
7,"Salesforce, Inc.",Delta Blue Carbon – 1,2250,2021,8460.0
8,"Salesforce, Inc.",Delta Blue Carbon – 1,2250,2021,8134.0
9,"Salesforce, Inc.",Delta Blue Carbon – 1,2250,2021,4624.0


In [14]:
# 💾 Optional: Export summaries
summary_df.to_csv("beneficiary_summary.csv", index=False)
buyer_projects.to_csv("salesforce_projects.csv", index=False)